### BELOW CODE IS TO WORK WITH SPARK IN COLAB, IGNORE IF NOT USING COLAB

In [1]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.7 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,691 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/re

In [2]:
import pyspark
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf
os.environ['PYSPARK_PYTHON']=sys.executable
os.environ['PYSPARK_DRIVER_PYTHON']=sys.executable

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('week9').getOrCreate()
spark

In [34]:
path='/content/AAME.csv'
df=spark.read.csv(path,header=True,inferSchema=True)
df.show(5)
df.printSchema()

+---------+----+----+----+-----+------+
|     Date|Open|High| Low|Close|Volume|
+---------+----+----+----+-----+------+
|31-Dec-13|4.04|4.13|3.96| 4.09| 30735|
|30-Dec-13|4.05|4.05|3.84|  3.9| 14646|
|27-Dec-13|4.02|4.05|3.99| 4.05|  5047|
|26-Dec-13|3.99|4.04|3.70| 4.01|  6309|
|24-Dec-13|3.90|3.97|3.84| 3.95| 13592|
+---------+----+----+----+-----+------+
only showing top 5 rows

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [35]:
from pyspark.sql.functions import col
df = df.withColumn("Open", col("Open").cast("double "))
df = df.withColumn("High", col("High").cast("double "))
df = df.withColumn("Low", col("Low").cast("double "))
df = df.withColumn("Volume", col("Volume").cast("double "))
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)



In [36]:
import pyspark.sql.functions as f
params={}
for col in df.columns:
  if col!='Date':
    mean=df.agg(f.avg(col)).collect()[0][0]
    std=df.agg(f.stddev(col)).collect()[0][0]
    params[col]=[mean,std]

In [37]:
for key,val in params.items():
  print(key,val)

Open [2.3776905972045754, 0.8594838332913094]
High [2.445060355781448, 0.855811640978281]
Low [2.3282719186785252, 0.8412114675601051]
Close [2.3474244152880743, 0.8579680474599332]
Volume [6616.119794637764, 14076.882853421579]


In [40]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import numpy as np
num_simulations = 1000

@udf(FloatType())
def open():
  return np.random.normal(params['Open'][0],params['Open'][0])

@udf(FloatType())
def High():
  return np.random.normal(params['High'][0],params['High'][0])

@udf(FloatType())
def Low():
  return np.random.normal(params['Low'][0],params['Low'][0])

@udf(FloatType())
def Close():
  return np.random.normal(params['Close'][0],params['Close'][0])

@udf(FloatType())
def Volume():
  return np.random.normal(params['Volume'][0],params['Volume'][0])

In [42]:
simulated_prices_df = df.select(
    open().alias("simulated_open"),
    Close().alias("simulated_close"),
    High().alias("simulated_high"),
    Low().alias("simulated_low"),
    Volume().alias("simulated_vol")
)

# Show the results
simulated_prices_df.show()

+--------------+---------------+--------------+-------------+-------------+
|simulated_open|simulated_close|simulated_high|simulated_low|simulated_vol|
+--------------+---------------+--------------+-------------+-------------+
|    -0.9782968|      -2.992069|     2.0054862|    4.8558536|    -606.2522|
|     4.1166506|      0.7854362|   -0.14350523|     3.158376|    5753.5415|
|    -2.4536436|      2.1556363|    0.98911667|    3.3869808|    15449.269|
|     1.2746093|      1.1008133|     3.6142232|    1.5298885|     9710.116|
|     3.5501034|      1.6232053|     1.4677219|   0.76851463|    7897.9414|
|     1.5931914|     -1.4981796|     1.5963776|   -0.6019404|     8555.023|
|     3.3889515|     -1.4126365|     1.9634925|     6.439457|     5667.803|
|     3.5240183|       3.972301|     4.7053895|    3.7281692|    12046.267|
|      5.535614|      2.7458217|     5.8277545|  0.029647453|    18265.121|
|     5.9172864|    -0.70438546|     -2.440493|   -0.9055956|     9480.559|
|     1.8193